In [1]:
import os
os.environ["XDG_CACHE_HOME"] = "/home/olab/tomerronen1/xdg_cache"
os.environ["AUTH_TOKEN"] = "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"

In [2]:
import sled
from transformers import AutoModelForSeq2SeqLM, AutoConfig
model_name = "tau/bart-base-sled-govreport"
auth_token = "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"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, use_auth_token=auth_token)

# from transformers.models.bart.modeling_bart import BartModel


There were unexpected keys in the checkpoint model loaded: ['_lm_head.weight'].


In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=auth_token)

In [4]:
import torch
from torch import Tensor


class GradCache:
    def __init__(self):
        self.cache = []

    def __call__(self, grad: Tensor) -> None:
        if (grad != 0).any():
            grad = grad.cpu().detach().clone()
            self.cache.append(grad)


In [5]:
from datasets import load_dataset
dataset = load_dataset("ccdv/govreport-summarization", split="validation[:10]")

No config specified, defaulting to: gov_report_summarization_dataset/document
Reusing dataset gov_report_summarization_dataset (/home/olab/tomerronen1/xdg_cache/huggingface/datasets/ccdv___gov_report_summarization_dataset)/document/1.0.0/57ca3042de9c40c218cc94084cbc80a99a161036134bfc88112c57d251443590)


In [56]:
batch = tokenizer(dataset[:2]["report"], return_tensors='pt', padding=True, truncation=True, max_length=200)
input_ids = batch["input_ids"]
attention_mask=batch["attention_mask"]
batch_size = input_ids.shape[0]
gen_batch = model.generate(**batch, num_beams=1, max_length=30)
decoder_attention_mask = (gen_batch != tokenizer.pad_token_id).long()
print(tokenizer.batch_decode(gen_batch))

/home/olab/tomerronen1/miniconda3/envs/mlskel/lib/python3.9/site-packages/sled/modeling_sled.py:693: UserWarning: prefix_length is missing from kwargs_tensor_keys (though expected for SLED prefix prepending)
  warnings.warn(f'{k} is missing from kwargs_tensor_keys (though expected for SLED prefix prepending)')


['</s><s>Part of the Mariana Islands Archipelago, the CNMI is a chain of 14 islands in the western Pacific Ocean, just north</s>', '</s><s>The U.S. pipeline network includes both interstate and intrastate pipelines, the vast majority of which fall into the latter category:</s>']


In [7]:
embedding_layer = model._underlying_model.model.shared
inputs_embeds = embedding_layer(input_ids)
inputs_embeds = inputs_embeds.unbind()
caches = []
for sequence_embeds in inputs_embeds:
    grad_cache = GradCache()
    sequence_embeds.register_hook(grad_cache)
    caches.append(grad_cache)
inputs_embeds = torch.stack(inputs_embeds)

In [8]:
model_output = model(inputs_embeds=inputs_embeds, attention_mask=attention_mask,
                     decoder_input_ids=gen_batch, decoder_attention_mask=decoder_attention_mask)

In [9]:
logits = model_output.logits[:,:-2,:]  # without additional token [-1], without the token that replaces the forced EOS [-2].
generated_tokens = gen_batch[:,1:-1]  # without the EOS that starts the generation, without the forced EOS at the end.
manual_greedy_output = logits.argmax(-1)
assert (manual_greedy_output == generated_tokens).all()

In [10]:
logprobs = logits.log_softmax(dim=-1)  # more correct, but then every word in the dictionary participates - is that good?
# logprobs = logits
generated_logprobs = torch.gather(input=logprobs, dim=-1, index=generated_tokens.unsqueeze(-1)).squeeze(-1)
manual_greedy_generated_logprobs = logprobs.max(dim=-1).values
assert (generated_logprobs == manual_greedy_generated_logprobs).all()

In [22]:
# i_target_token = 6
# i_target_token = 7
# i_target_token = "entire_sequence"
i_target_token = "all_individual_target_tokens"
if i_target_token == "all_individual_target_tokens":
    for i_example in range(batch_size):
        for i_token in range(generated_logprobs.shape[1]):
            generated_logprobs[i_example][i_token].backward(retain_graph=True)
            model.zero_grad()
else:
    if i_target_token == "entire_sequence":
        logprobs_to_derivate = generated_logprobs.sum(dim=-1)
        for gen_seq in tokenizer.batch_decode(generated_tokens):
            print(gen_seq)
    else:
        logprobs_to_derivate = generated_logprobs[:,i_target_token]
        print(tokenizer.convert_ids_to_tokens(generated_tokens[:,i_target_token]))

    for i in range(batch_size):
        logprobs_to_derivate[i].backward(retain_graph=True)
        model.zero_grad()


In [35]:
grads = torch.stack([torch.stack(grad_cache.cache) for grad_cache in caches]) # bsz, taget tokens, input tokens, embed dim
grads.shape
    

torch.Size([2, 28, 200, 768])

In [70]:
# from functools import reduce
saliency = grads.abs().sum(dim=-1)  # abd grad method. bsz, target tokens, input tokens
# saliency = (grads * inputs_embeds.unsqueeze(1)).sum(dim=-1)  # grad dot input method. bsz, target tokens, input tokens
saliency = saliency - saliency.min(dim=-1, keepdims=True).values
saliency = saliency / saliency.max(dim=-1, keepdims=True).values
# saliency = grad_dot_input.softmax(dim=-1)

In [71]:
import pandas as pd
import matplotlib.pyplot as plt

cols = {}
i_example = 0
for i_target_token in range(saliency.shape[1]):
    input_tokens = tokenizer.convert_ids_to_tokens(input_ids[i_example])
    input_tokens = pd.Series(input_tokens).str.replace('Ġ', '_').replace('<s>', 'BOS')
    target_token = tokenizer.convert_ids_to_tokens([generated_tokens[i_example][i_target_token]])[0].replace('Ġ', '_').replace('<s>', 'BOS')
    cols[f"Input Tokens_{target_token}"] = input_tokens
    cols[f"Saliency_{target_token}"] = saliency[i_example][i_target_token].detach().cpu().numpy()

saliency_df = pd.DataFrame(cols)
cmap = plt.cm.get_cmap("coolwarm")
saliency_df = saliency_df.style.background_gradient(cmap=cmap)

print(tokenizer.decode(input_ids[i_example]))
print()
print(tokenizer.decode(generated_tokens[i_example]))
saliency_df

<s>Part of the Mariana Islands Archipelago, the CNMI is a chain of 14 islands in the western Pacific Ocean, just north of Guam and about 3,200 miles west of Hawaii. The CNMI has a total population of 53,890, according to the CNMI’s 2016 Household, Income, and Expenditures Survey. Almost 90 percent of the population (48,200) resided on the island of Saipan, with an additional 6 percent (3,056) on the island of Tinian and 5 percent (2,635) on the island of Rota. The Consolidated Natural Resources Act of 2008 amended the U.S.– CNMI covenant to apply federal immigration law to the CNMI after a transition period. To provide for an orderly transition from the CNMI immigration system to the U.S. federal immigration system under the immigration laws of the United States, DHS established the CW program in 2011. Under the program, foreign workers are</s>

<s>Part of the Mariana Islands Archipelago, the CNMI is a chain of 14 islands in the western Pacific Ocean, just north


,Input Tokens_BOS,Saliency_BOS,Input Tokens_Part,Saliency_Part,Input Tokens__of,Saliency__of,Input Tokens__the,Saliency__the,Input Tokens__Mar,Saliency__Mar,Input Tokens_iana,Saliency_iana,Input Tokens__Islands,Saliency__Islands,Input Tokens__Arch,Saliency__Arch,Input Tokens_ipel,Saliency_ipel,Input Tokens_ago,Saliency_ago,"Input Tokens_,","Saliency_,",Input Tokens__CN,Saliency__CN,Input Tokens_MI,Saliency_MI,Input Tokens__is,Saliency__is,Input Tokens__a,Saliency__a,Input Tokens__chain,Saliency__chain,Input Tokens__14,Saliency__14,Input Tokens__islands,Saliency__islands,Input Tokens__in,Saliency__in,Input Tokens__western,Saliency__western,Input Tokens__Pacific,Saliency__Pacific,Input Tokens__Ocean,Saliency__Ocean,Input Tokens__just,Saliency__just,Input Tokens__north,Saliency__north
0,BOS,0.085122,BOS,0.070439,BOS,0.052430,BOS,0.071219,BOS,0.038729,BOS,0.033355,BOS,0.050380,BOS,0.045850,BOS,0.055940,BOS,0.037797,BOS,0.110296,BOS,0.078970,BOS,0.033215,BOS,0.099814,BOS,0.080886,BOS,0.027175,BOS,0.040518,BOS,0.041598,BOS,0.087004,BOS,0.027513,BOS,0.027498,BOS,0.027299,BOS,0.032949,BOS,0.020528
1,Part,0.061851,Part,1.000000,Part,0.169114,Part,0.240309,Part,0.183055,Part,0.112345,Part,0.116589,Part,0.188844,Part,0.140281,Part,0.121070,Part,0.473377,Part,0.372614,Part,0.066818,Part,0.445643,Part,0.428088,Part,0.081927,Part,0.095073,Part,0.082854,Part,0.351784,Part,0.071316,Part,0.036879,Part,0.050981,Part,0.111512,Part,0.043504
2,_of,0.017627,_of,0.262942,_of,0.087012,_of,0.114008,_of,0.098498,_of,0.069717,_of,0.057213,_of,0.087710,_of,0.066452,_of,0.067511,_of,0.159450,_of,0.109332,_of,0.027254,_of,0.180077,_of,0.176996,_of,0.034989,_of,0.041231,_of,0.035950,_of,0.166289,_of,0.033937,_of,0.018167,_of,0.025366,_of,0.040631,_of,0.016708
3,_the,0.022387,_the,0.091401,_the,0.075393,_the,0.087305,_the,0.198171,_the,0.130151,_the,0.124239,_the,0.077568,_the,0.067456,_the,0.052315,_the,0.134045,_the,0.100164,_the,0.022475,_the,0.130073,_the,0.149820,_the,0.020291,_the,0.037911,_the,0.034724,_the,0.159128,_the,0.035894,_the,0.020376,_the,0.023963,_the,0.029399,_the,0.014382
4,_Mar,0.078854,_Mar,0.110276,_Mar,0.086957,_Mar,0.165886,_Mar,1.000000,_Mar,0.509290,_Mar,0.274150,_Mar,0.152297,_Mar,0.139948,_Mar,0.111426,_Mar,0.190632,_Mar,0.362941,_Mar,0.064911,_Mar,0.245207,_Mar,0.189692,_Mar,0.033751,_Mar,0.055716,_Mar,0.054720,_Mar,0.227147,_Mar,0.133978,_Mar,0.109034,_Mar,0.038208,_Mar,0.049942,_Mar,0.024587
5,iana,0.022801,iana,0.063645,iana,0.073876,iana,0.123162,iana,0.515039,iana,1.000000,iana,0.507650,iana,0.139087,iana,0.100687,iana,0.087819,iana,0.114771,iana,0.296517,iana,0.047424,iana,0.131278,iana,0.129576,iana,0.022599,iana,0.050417,iana,0.050772,iana,0.123508,iana,0.061046,iana,0.060961,iana,0.032074,iana,0.034734,iana,0.021066
6,_Islands,0.043749,_Islands,0.079212,_Islands,0.156144,_Islands,0.207221,_Islands,0.299419,_Islands,0.326517,_Islands,1.000000,_Islands,0.277593,_Islands,0.232503,_Islands,0.131548,_Islands,0.220819,_Islands,0.218994,_Islands,0.069366,_Islands,0.225688,_Islands,0.235207,_Islands,0.118712,_Islands,0.077490,_Islands,0.121803,_Islands,0.219440,_Islands,0.070607,_Islands,0.086393,_Islands,0.059589,_Islands,0.053525,_Islands,0.033516
7,_Arch,0.035805,_Arch,0.105094,_Arch,0.101966,_Arch,0.150010,_Arch,0.206812,_Arch,0.168873,_Arch,0.306972,_Arch,1.000000,_Arch,0.610458,_Arch,0.245818,_Arch,0.229029,_Arch,0.212077,_Arch,0.052656,_Arch,0.288792,_Arch,0.180101,_Arch,0.052918,_Arch,0.075803,_Arch,0.073306,_Arch,0.184230,_Arch,0.049274,_Arch,0.040281,_Arch,0.046665,_Arch,0.059410,_Arch,0.028426
8,ipel,0.038423,ipel,0.082425,ipel,0.177572,ipel,0.198364,ipel,0.176472,ipel,0.150037,ipel,0.222915,ipel,0.359973,ipel,1.000000,ipel,0.585516,ipel,0.259609,ipel,0.245102,ipel,0.057163,ipel,0.299978,ipel,0.236143,ipel,0.093838,ipel,0.089289,ipel,0.118249,ipel,0.188725,ipel,0.057445,ipel,0.056463,ipel,0.070086,ipel,0.059158,ipel,0.032526
9,ago,0.014873,ago,0.044038,ago,0.183078,ago,0.082474,ago,0.077832,ago,0.063497,ago,0.099311,